# CLICK_SERCH
## Generacion de cliques
+ Se generan con biopandas para obtener los atomos de $C_\alpha$ y sus coordenadas.
+ Se calcula la distancia y se genera un grafo completo con la distancia entre cada par de atomos.
+ Se restringen los enlaces por una distancia dada y se generan los cliques que tengas un numero k de elementos para pertencer al clique.
+ Una ves generados los cliques de cada proteina se extraen sus coordenadas para poderlas comparar

In [243]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('max_rows', 200)
pd.set_option('max_columns', 40)

import networkx as nx,community
import matplotlib.pyplot as plt
plt.style.use('ggplot')
font = {'family' : 'sans',
        'weight' : 'bold',
        'size'   : 20}
plt.rc('font', **font)

plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['ytick.labelsize'] = 16
plt.rcParams[u'figure.figsize'] = (16,8)

In [244]:
#mas librerias que voy obteniendo
import biopandas.pdb as bp
biop = bp.PandasPdb() #libreria de lectura de pdbs
#libreria de calculo de distancia euclidiana
from scipy.spatial.distance import pdist, squareform
#libreria de mate
import numpy as np
import itertools as it
#Libreria de MA para RMSD
import sys
sys.path.append('math_tricks/')
import math_vect_tools as mvt
#Libreria de graficacion interactiva
import plotly.plotly as py
import plotly.graph_objs as go

In [245]:
# generar mismos resultados que el algoritmo click
path1 ='/Users/serch/pdbmani/Serch/1phr.pdb'
path2 ='/Users/serch/pdbmani/Serch/1tig.pdb'

#funcion de lectura con biopandas
def read_biopdb(path):
        df = biop.read_pdb(path)
        df_atom = df.df['ATOM']
        #OJO AQUI ESTA ADECUADO AL PDB   para elegir solo un frame en trj_0 y trj_0_A [:1805]
        df_ca = df_atom[df_atom.atom_name == 'CA'][[
        'atom_number','atom_name','residue_name','residue_number',
        'x_coord','y_coord','z_coord']]
        columna_vector = []
        for i in zip(df_ca.x_coord.tolist(),df_ca.y_coord.tolist(),df_ca.z_coord.tolist()):
            columna_vector.append(list(i))
            
        df_ca['vector'] = columna_vector
        return(df_ca)

In [246]:
df_ca1 = read_biopdb(path1)
df_ca2 = read_biopdb(path2)

In [247]:
# primer RMSD entre atomos
a = df_ca1.vector.values[0]
b = df_ca2.vector.values[0]
values_diff = []
for i,val1 in enumerate(a):
    for j,val2 in enumerate(b):
        if i == j:
            diff = val1-val2
            diff_2 = diff**2
            values_diff.append(diff_2)

sum(values_diff)/2


1613.098278

In [248]:
# #se calcula la distancia entre cada par de nodos.
# # def distancia_entre_atomos(df_ca):
# distancias = []
# #se calcula la distancia euclidiana entre cada atomo de carbon alfalfa
# for v,i in zip(df_ca1.vector,df_ca1.atom_number):
#     distancia_un_atomo = []
#     for av,j in zip(df_ca1.vector,df_ca1.atom_number):
#         distancia = pdist([v,av],metric='euclidean').item()
#         distancia_un_atomo.append(distancia)
#     distancias.append(distancia_un_atomo)
    
# pd.DataFrame(index=df_ca1.atom_number,
#                                       columns=df_ca1.atom_number,
#                                       data=distancias)

In [249]:
#se calcula la distancia entre cada par de nodos.
def distancia_entre_atomos(df_ca):
    distancias = []
    #se calcula la distancia euclidiana entre cada atomo de carbon alfalfa
    for v,i in zip(df_ca.vector,df_ca.atom_number):
        distancia_un_atomo = []
        for av,j in zip(df_ca.vector,df_ca.atom_number):
            distancia = pdist([v,av],metric='euclidean').item()
            distancia_un_atomo.append(distancia)
        distancias.append(distancia_un_atomo)
    #se genera la matriz de adyacencias para la red
    df_da = pd.DataFrame(index=df_ca.atom_number,columns=df_ca.atom_number,data=distancias)
    return(df_da)

In [250]:
df_da1 = distancia_entre_atomos(df_ca1)
df_da2 = distancia_entre_atomos(df_ca2)

In [251]:
def gen_3_cliques(df_da, dth = 10, k=3):
    """Genera n-cliques de dataframe de distancias, tomando en cuenta los enlaces menores o iguales
    a dth y forma los k-cliques que elijas 
    valores por default:
    dth=10, k=3
    Te devuelve un df con los valores de los cliques y 
    un resumen de como es la red antes y despues del filtro"""
    #red de distancias completa
    red = nx.from_pandas_adjacency(df_da)
    print("red antes de filtros:",nx.info(red))

    #filtro de distancias
    edgesstrong = [(u,v) for (u,v,d) in red.edges(data=True) if d["weight"] <= dth]

    red = nx.Graph(edgesstrong)
    print("=="*20)
    print("red despues de filtros:",nx.info(red))

    n_cliques = [clq for clq in nx.find_cliques(red) if len(clq) >=k]
    print('numero de cliques maximos encontrados:',len(n_cliques))

    lista_cliques = []
    for i,v in enumerate(n_cliques):
        a = list(it.combinations(v,k))
        for j in a:
            if set(j) not in lista_cliques:
                #recuerda que para comparar elementos utiliza set, y apilalos como set
                lista_cliques.append(set(j))

    df_lc = pd.DataFrame(lista_cliques)            
    print("numero de %s-cliques posibles:" % (k), df_lc.shape[0])
    return(df_lc)

In [252]:
df_lc1 = gen_3_cliques(df_da1)
print('--'*50)
df_lc2 = gen_3_cliques(df_da2)

red antes de filtros: Name: 
Type: Graph
Number of nodes: 154
Number of edges: 11781
Average degree: 153.0000
red despues de filtros: Name: 
Type: Graph
Number of nodes: 154
Number of edges: 1378
Average degree:  17.8961
numero de cliques maximos encontrados: 419
numero de 3-cliques posibles: 4480
----------------------------------------------------------------------------------------------------
red antes de filtros: Name: 
Type: Graph
Number of nodes: 88
Number of edges: 3828
Average degree:  87.0000
red despues de filtros: Name: 
Type: Graph
Number of nodes: 88
Number of edges: 709
Average degree:  16.1136
numero de cliques maximos encontrados: 246
numero de 3-cliques posibles: 2102


In [253]:
# #red de distancias completa
# red = nx.from_pandas_adjacency(df_da1)
# print(nx.info(red))


# #filtro de distancias
# edgesstrong = [(u,v) for (u,v,d) in red.edges(data=True) if d["weight"] <= 10]

# red = nx.Graph(edgesstrong)
# print("=="*20)
# print(nx.info(red))

# cliques3 = [clq for clq in nx.find_cliques(red) if len(clq) >=3]
# print('numero de cliques maximos encontrados:',len(cliques3))

# lista_cliques = []
# for i,v in enumerate(cliques3):
#     a = list(it.combinations(v,3))
#     for j in a:
#         if set(j) not in lista_cliques:
#             #recuerda que para comparar elementos utiliza set, y apilalos como set
#             lista_cliques.append(set(j))

# print("numero de 3-cliques posibles:",pd.DataFrame(lista_cliques).shape[0])

In [254]:
###CHECK DE NUMERO DE CLIQUES CORRECTO####
#se genera la matriz de adyacencias para la red
# distancias_adyacenctes = pd.DataFrame(index=df_ca1.atom_number,
#                                       columns=df_ca1.atom_number,
#                                       data=distancias)

# #red de distancias completa
# red = nx.from_pandas_adjacency(distancias_adyacenctes)
# print(nx.info(red))

# print("=="*20)
# print(nx.info(red))
# cliques3 = [clq for clq in nx.find_cliques(red) if len(clq) >=3]
# print('numero de cliques maximos encontrados:',len(cliques3))

# lista_cliques = []
# for i,v in enumerate(cliques3):
#     a = list(it.combinations(v,3))
#     for j in a:
#         #if set(j) not in lista_cliques:
#             #recuerda que para comparar elementos utiliza set, y apilalos como set
#         lista_cliques.append(set(j))

# print("numero de 3-cliques posibles:",pd.DataFrame(lista_cliques).shape[0])
###CHECK DE NUMERO DE CLIQUES CORRECTO####

In [255]:
def get_coord_clique(df_ca,df_lc):
    lista_matriz_coordendas = []
    for i in df_lc.index:
        mat_dist = [df_ca[df_ca.atom_number==df_lc.iloc[i,0]].vector.values[0],
                df_ca[df_ca.atom_number==df_lc.iloc[i,1]].vector.values[0],
                df_ca[df_ca.atom_number==df_lc.iloc[i,2]].vector.values[0]]
        lista_matriz_coordendas.append(mat_dist)

    df_lc['matriz_coordenadas'] = lista_matriz_coordendas
    return(df_lc)

In [256]:
df_lc1 = get_coord_clique(df_ca1,df_lc1)
df_lc2 = get_coord_clique(df_ca2,df_lc2)

In [257]:
df_lc1.head()

,0,1,2,matriz_coordenadas
0,121,514,431,"[[4.736, 31.117, 32.112], [-4.239, 29.134, 31...."
1,121,514,507,"[[4.736, 31.117, 32.112], [-4.239, 29.134, 31...."
2,121,514,393,"[[4.736, 31.117, 32.112], [-4.239, 29.134, 31...."
3,121,514,387,"[[4.736, 31.117, 32.112], [-4.239, 29.134, 31...."
4,514,507,431,"[[-4.239, 29.134, 31.826], [-1.742, 27.922, 34..."


## Comparacion de Cliques
The objective is to compute a one to one mapping between amino acid residues of the two structures
A and B. To begin with, all possible 3-body cliques $A_3$ and $B_3$, where $A_3$ and $B_3  S_3$, are compared to one another (inclusive of all permutations). Equivalent pairs of cliques are deduced according to the relations in Equations (2–4). A pair of $(A_3, B_3)$ is matched if their RMSD on superimposition is smaller than a preset threshold ($RMSD_3=0.15 A˚$). RMSD between cliques is
calculated by 3-D least squares fit (39). 

Additionally, amino acid residue secondary-structure state and side chain solvent accessible area also determines what pair of cliques are matched. Secondary structure provides the general three-dimensional form of local segments of proteins while side-chain solvent accessibility is the degree to which a residue in a protein is accessible to a solvent molecule. For matching of a pair of cliques in our algorithm, the secondary-structure score between two equivalent residues $A_i$ and $B_j$ are compared
SSM is an empirically determined secondary-structure match matrix (Table 1), $SS(A_i)$ is the secondary-structure state of amino acid residue $A_i$, and $s$ is a preset threshold for matching secondary-structure elements. 

The cut-off threshold for comparing secondary structure used in this study was 2, hence $SSM[Ai,Bj]< 2 [Equation (2)]$. This implies that, either residues of regular secondary structures can only match with other residues of the same secondary structure, or with residues in loops.
The solvent accessibility score between two residues $A_i$ and $B_j$ from solvent accessibility matrix (Table 2) are matched by using the inequality [Equation (3)]: SAM is an empirical solvent accessibility match matrix (Table 2), $SA(A_i)$ is the side-chain solvent accessibility of amino acid residue $A_i$, and a is a preset threshold for matching solvent accessible area states. The cut-off threshold for solvent accessibility matching is $a=1$, implying that residues categorized in different accessible area
classes cannot be matched. However, this criterion is relaxed to allow the matching of two residues in
adjacent accessible area classes if their side chain accessible areas are within 10% of each other.
Next, $A_3$ and $B_3$ are extended to 4-body cliques $A_4$ and $B_4$, by including one residue, $A_i$ and $B_j$ respectively, subject to the distance threshold criterion [Equation (1)].

This new pair $(A_i, B_j)$ and matched residues of $(A_3, B_3)$ are used to superimpose the pair of cliques $A_4=A_3[A_i$ and $B_4=B_3[B_j$. Pairs of four-body cliques, $A_4$ and $B_4$, are matched if their RMSD is smaller than another preset threshold, $RMSD_4=0.30A˚ [Equation (4), n=4]$. 

Pairs of n body cliques, $A_n=A_n1[A_i and B_n=B_n1[ B_j$ are selected if their RMSD is smaller than a preset threshold RMSDn (Table 3). Every value of n has a different RMSD threshold, RMSDn. See the section on RMSD threshold optimization for details. At every step the secondary structure and accessible area comparisons [Equations (2) and (3)] are also performed.

All matched pairs of 4-body cliques $A_4$ and $B_4$ are extended to all possible higher order cliques, $A_n$ and $B_n$, where $A_n, B_n S_n$ and $n>4$. In this study, cliques are extended to a maximum of seven constituent residues.

### Pasos para comparar
Para obtener el __RMSD__ es necesario primero rotar y trasladar al origen ambos atomos o moleculas para generar su __RMSD__

Para obtener C, $\alpha$, $\beta$ con:
   + $\Phi$
   + $\Psi$
1. Matriz de comparacion de Estructura Secundaria (SSM)
2. Solvente Accesible (SAM)

In [258]:
# # prueba de baricentro utilizando el centro de un circulo que pasa por los tres puntos
# # una ves que tenemos los cliques calculamos su centro geometrico para poder calcular el RMSD
# #primero obtenemos el array de cada punto
# algo = []
# for i in range(df_lc1.shape[0]):
#      if i < 3:
#         A = np.array(df_lc1.matriz_coordenadas[i][0])
#         B = np.array(df_lc1.matriz_coordenadas[i][1])
#         C = np.array(df_lc1.matriz_coordenadas[i][2])
        
#         #calculamos su distancia entre cada punto
#         a = np.linalg.norm(C - B)
#         b = np.linalg.norm(C - A)
#         c = np.linalg.norm(B - A)
#         print(a,b,c)
#         #se calcula s que es 
#         s = (a + b + c) / 2
        
# #         #se calcula el radio R
# #         R = a*b*c / 4 / np.sqrt(s * (s - a) * (s - b) * (s - c))

#         #se calculan las coordenadas baricentricas
#         b1 = a*a * (b*b + c*c - a*a)
#         b2 = b*b * (a*a + c*c - b*b)
#         b3 = c*c * (a*a + b*b - c*c)

#         #se pasa a coordenadas centricas de los puntos
#         P = np.column_stack((A, B, C)).dot(np.hstack((b1, b2, b3)))

#         P /= b1 + b2 + b3
        
#         algo.append(P)
#         print(P)
# #         R = round(R,8)
# print(len(algo))

9.822362648568827 8.746098673122777 9.195907241811435
[ 0.84303666 27.43120633 31.99283857]
3.8045179720958058 7.585272902143994 9.195907241811435
[ 0.14994365 30.69371327 31.12206406]
8.476275715194735 6.470443261477534 9.195907241811435
[ 0.45010049 29.3552926  30.98283513]
3


In [259]:
#Si esta bien el promedio de los puntos
def baricenter_clique(df_lc):
    coord_center = []
    for i in range(df_lc.shape[0]):
        A = np.array(df_lc.matriz_coordenadas[i][0])
        B = np.array(df_lc.matriz_coordenadas[i][1])
        C = np.array(df_lc.matriz_coordenadas[i][2])

        x1 = round((A[0]+B[0]+C[0])/3,5)
        y1 = round((A[1]+B[1]+C[1])/3,5)
        z1 = round((A[2]+B[2]+C[2])/3,5)

        coord_center.append([x1,y1,z1])


    df_lc['baricentro_clique'] = coord_center
    return df_lc

In [260]:
df_lc1 = baricenter_clique(df_lc1)
df_lc2 = baricenter_clique(df_lc2)
df_lc1.head()

,0,1,2,matriz_coordenadas,baricentro_clique
0,121,514,431,"[[4.736, 31.117, 32.112], [-4.239, 29.134, 31....","[1.17733, 27.59633, 32.00333]"
1,121,514,507,"[[4.736, 31.117, 32.112], [-4.239, 29.134, 31....","[-0.415, 29.391, 32.78867]"
2,121,514,393,"[[4.736, 31.117, 32.112], [-4.239, 29.134, 31....","[1.08367, 29.078, 30.49467]"
3,121,514,387,"[[4.736, 31.117, 32.112], [-4.239, 29.134, 31....","[-0.009, 29.31467, 29.84167]"
4,514,507,431,"[[-4.239, 29.134, 31.826], [-1.742, 27.922, 34...","[-0.982, 26.53133, 32.77533]"


In [261]:
# PLOT DE COMO SE VEN LOS ATOMOS AL COMPARAR
trace1 = go.Scatter3d(
    x=df_lc1.matriz_coordenadas[0][:][0],
    y=df_lc1.matriz_coordenadas[0][:][1],
    z=df_lc1.matriz_coordenadas[0][:][2],
    mode='markers+lines',
    marker=dict(
        size=12,
        line=dict(
            color='rgba(217, 217, 217, 0.14)',
            width=0.5
        ),
        opacity=0.8
    )
)

trace2 = go.Scatter3d(
    x=df_lc2.matriz_coordenadas[0][:][0],
    y=df_lc2.matriz_coordenadas[0][:][1],
    z=df_lc2.matriz_coordenadas[0][:][2],
    mode='markers+lines',
    marker=dict(
        color='rgb(127, 127, 127)',
        size=12,
        symbol='circle',
        line=dict(
            color='rgb(204, 204, 204)',
            width=1
        ),
        opacity=0.9
    )
)
data = [trace1,trace2]
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='simple-3d-scatter')

In [264]:
def center_vectors(df_lc):
    vectores_centricos = []
    for i,val in enumerate(df_lc.baricentro_clique):
    #     extraccion de coordenadas de cada atomo
        A = np.array(df_lc.matriz_coordenadas[i][0])
        B = np.array(df_lc.matriz_coordenadas[i][1])
        C = np.array(df_lc.matriz_coordenadas[i][2])
        #calculo de vectores
        vec_a = np.round(list(A - val),6)
        vec_b = np.round(list(B - val),6)
        vec_c = np.round(list(C - val),6)

        vectores_centricos.append([vec_a,vec_b,vec_c])

    df_lc['vec_center'] = vectores_centricos
    return(df_lc)

In [267]:
df_lc1 = center_vectors(df_lc1)
df_lc2 = center_vectors(df_lc2)

In [285]:
df_lc1.vec_center[0],df_lc2.vec_center[0]

([array([3.55867, 3.52067, 0.10867]),
  array([-5.41633,  1.53767, -0.17733]),
  array([ 1.85767, -5.05833,  0.06867])],
 [array([-1.19667, -0.69   , -2.02333]),
  array([-0.53567, -0.872  ,  3.06667]),
  array([ 1.73233,  1.562  , -1.04333])])

In [377]:
#funcion para obtener los valores de la prerotacion
def R_ij(i,j,a1=0,a2=0):
    """Recuerda que 0-->1,1-->2,2-->2 en los indices de R
    a1,a2 corresponden a que atomo quieren que se compare este valor es 
    para generar las permutaciones y compare sobre todos los a2  de a1[0] y asi >)"""
    # por si me equivoco en los indices xD o podria hacer un
    # diccionario de valores para que no tenga fallas
    dict_convencion = {1:0,2:1,3:2}
    i = dict_convencion.get(i)
    j = dict_convencion.get(j)
    
    values = []
    for k in range(3):
        atom_value1 = df_lc1.vec_center[a1][k][i]
        atom_value2 = df_lc2.vec_center[a2][k][j]
        value = atom_value1 * atom_value2
        values.append(value)
        valor = sum(values)

    return(valor)

In [383]:
R_ij(1,3)

-25.7486233333

In [405]:
#generando la matriz gigante, para generar la funcion hay que agregar los valores de a1 y a2
#primer renglon
R11R22R33 = (R_ij(1,1,a1=0,a2=0) + R_ij(2,2,a1=0,a2=0) + R_ij(3,3,a1=0,a2=0))
R23_R32 = (R_ij(2,3,a1=0,a2=0) - R_ij(3,2,a1=0,a2=0))
R31_R13 = (R_ij(3,1,a1=0,a2=0) - R_ij(1,3,a1=0,a2=0))
R12_R21 = (R_ij(1,2,a1=0,a2=0) - R_ij(2,1,a1=0,a2=0))
#segundo renglon
R11_R22_R33 = (R_ij(1,1,a1=0,a2=0) - R_ij(2,2,a1=0,a2=0) - R_ij(3,3,a1=0,a2=0))
R12R21 = (R_ij(1,2,a1=0,a2=0) + R_ij(2,1,a1=0,a2=0))
R13R31 = (R_ij(1,3,a1=0,a2=0) - R_ij(3,1,a1=0,a2=0))
#tercer renglon
_R11R22_R33 = (-R_ij(1,1,a1=0,a2=0) + R_ij(2,2,a1=0,a2=0) - R_ij(3,3,a1=0,a2=0))
R23R32 = (R_ij(2,3,a1=0,a2=0) + R_ij(3,2,a1=0,a2=0))
#cuarto renglon
_R11_R22R33 = (-R_ij(1,1,a1=0,a2=0) - R_ij(2,2,a1=0,a2=0) + R_ij(3,3,a1=0,a2=0))

matriz_gigante = np.round([
    [R11R22R33, R23_R32 , R31_R13, R12_R21],
    [R23_R32, R11_R22_R33, R12R21, R13R31],
    [R31_R13, R12R21, _R11R22_R33, R23R32],
    [R12_R21, R13R31, R23R32, _R11_R22R33]
],6)

In [406]:
matriz_gigante

array([[-10.645646,   2.682645,  25.832531,  18.968699],
       [  2.682645,  14.367465,  -8.630223, -25.832531],
       [ 25.832531,  -8.630223, -12.696798,   3.056469],
       [ 18.968699, -25.832531,   3.056469,   8.974979]])

In [416]:
#eigenvectores y eigenvalores
eignvalues,eigenvectors = np.linalg.eig(matriz_gigante)
#eigenvector maximo y eigenvalor checar si el vector esta bien la parte del [:,blabla]
print(eigenvectors[:,np.argmax(eignvalues)])
print(max(eignvalues))

[ 0.32795402 -0.62000923  0.27837801  0.65615578]


44.161931946266336

In [442]:
#comienza el calculo rmsd 
#algo asi seria 
e = np.sqrt((np.dot(df_lc1.vec_center[0][0],df_lc1.vec_center[0][0]) + np.dot(
    df_lc2.vec_center[0][0],df_lc2.vec_center[0][0])) - (
    2 * max(eignvalues))/3)
e

1.2774012683266152